In [1]:
import cv2 as cv 
import numpy as np
import pandas as pd

import os
import glob

In [2]:
X = []
Y = []
DIR = r"E:\Dataset"

classifier = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

In [3]:
from scipy.signal import butter, filtfilt
import numpy as np

def temporal_bandpass_filter(signal, fs, low=0.7, high=4.0):
    nyq = fs / 2
    low /= nyq
    high /= nyq
    b, a = butter(3, [low, high], btype='band')
    return filtfilt(b, a, signal)

def amplify_eulerian(signal, fs, alpha=50):
    filtered = temporal_bandpass_filter(signal, fs)
    amplified = filtered * alpha
    return signal + amplified  # Enhanced signal

In [4]:
# def extract_face_roi_means(frame):
#     gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     green = frame[:, :, 1]  # Extract green channel

#     faces = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
#     if len(faces) == 0:
#         return None

#     x, y, w, h = faces[0]  # Use the first face

#     # Define ROIs based on face box proportions
#     roi_means = 0

#     # Forehead region (top center)
#     fh_x1 = x + int(0.3 * w)
#     fh_x2 = x + int(0.7 * w)
#     fh_y1 = y + int(0.05 * h)
#     fh_y2 = y + int(0.25 * h)
#     forehead = green[fh_y1:fh_y2, fh_x1:fh_x2]
#     roi_means+= np.mean(forehead)

#     # Left cheek
#     lc_x1 = x + int(0.05 * w)
#     lc_x2 = x + int(0.3 * w)
#     lc_y1 = y + int(0.5 * h)
#     lc_y2 = y + int(0.75 * h)
#     left_cheek = green[lc_y1:lc_y2, lc_x1:lc_x2]
#     roi_means+= np.mean(left_cheek)

#     # Right cheek
#     rc_x1 = x + int(0.7 * w)
#     rc_x2 = x + int(0.95 * w)
#     rc_y1 = lc_y1
#     rc_y2 = lc_y2
#     right_cheek = green[rc_y1:rc_y2, rc_x1:rc_x2]
#     roi_means+= np.mean(right_cheek)

#     # Chin
#     ch_x1 = x + int(0.4 * w)
#     ch_x2 = x + int(0.6 * w)
#     ch_y1 = y + int(0.8 * h)
#     ch_y2 = y + int(0.95 * h)
#     chin = green[ch_y1:ch_y2, ch_x1:ch_x2]
#     roi_means+= np.mean(chin)

#     return roi_means/4


In [5]:
# for x in os.listdir(DIR):
#     folder_path = os.path.join(DIR, x)
#     if not os.path.isdir(folder_path):
#         continue

#     video_list = glob.glob(os.path.join(folder_path, '*.avi'))
#     csv_list = glob.glob(os.path.join(folder_path, '*.csv'))

#     if not video_list or not csv_list:
#         continue

#     video = video_list[0]
#     csv = csv_list[0]

#     cap = cv.VideoCapture(video)
#     df = pd.read_csv(csv)

#     fps = cap.get(cv.CAP_PROP_FPS)
    
#     X_feature = []
#     Y_feature = []

#     total_rows = len(df)
#     usable_rows = total_rows - (total_rows % 24)

#     for i in range(usable_rows):
#         timestamp_ms = df["Timestep (ms)"][i]
#         frame_number = int((timestamp_ms / 1000.0) * fps)
#         cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         mean_g = np.mean(frame[:,:,1])

#         X_feature.append(mean_g)
#         Y_feature.append(df["Heart rate (HR)"][i])

#     b = int(len(X_feature) / 24)
#     print(f"Processing {x}: {b} batches of 24 frames")
    
#     X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)   
#     Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

#     X.append(X_feature)
#     Y.append(Y_feature)


In [6]:
## use of bandpassfilter and amplify_eulerian
for x in os.listdir(DIR):
    folder_path = os.path.join(DIR, x)
    if not os.path.isdir(folder_path):
        continue

    video_list = glob.glob(os.path.join(folder_path, '*.avi'))
    csv_list = glob.glob(os.path.join(folder_path, '*.csv'))

    if not video_list or not csv_list:
        continue

    video = video_list[0]
    csv = csv_list[0]

    cap = cv.VideoCapture(video)
    df = pd.read_csv(csv)

    fps = cap.get(cv.CAP_PROP_FPS)
    
    X_feature = []
    Y_feature = []

    total_rows = len(df)
    usable_rows = total_rows - (total_rows % 24)

    for i in range(usable_rows):
        timestamp_ms = df["Timestep (ms)"][i]
        frame_number = int((timestamp_ms / 1000.0) * fps)
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

        ret, frame = cap.read()
        if not ret:
            break
        
        mean_g = np.mean(frame[:,:,1])

        X_feature.append(mean_g)
        Y_feature.append(df["Heart rate (HR)"][i])

    b = int(len(X_feature) / 24)
    print(f"Processing {x}: {b} batches of 24 frames")

    X_signal=temporal_bandpass_filter(X_feature, 25, low=0.7, high=4.0)
    X_amplified=amplify_eulerian(X_signal, 25, alpha=50)
    
    X_feature = np.array(X_amplified[:b*24]).reshape(b, 24, 1)   
    Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

    X.append(X_feature)
    Y.append(Y_feature)


Processing 11-gt: 207 batches of 24 frames
Processing 12-gt: 215 batches of 24 frames
Processing 5-gt: 217 batches of 24 frames
Processing 6-gt: 215 batches of 24 frames
Processing 7-gt: 213 batches of 24 frames
Processing 8-gt: 189 batches of 24 frames
Processing after-exercise: 304 batches of 24 frames


In [7]:
# ##Mediapipe feature extraction 1

# import os
# import glob
# import cv2 as cv
# import numpy as np
# import pandas as pd
# import mediapipe as mp

# X = []
# Y = []

# mp_face_detection = mp.solutions.face_detection
# mp_drawing = mp.solutions.drawing_utils

# with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:

#     for x in os.listdir(DIR):
#         folder_path = os.path.join(DIR, x)
#         if not os.path.isdir(folder_path):
#             continue

#         video_list = glob.glob(os.path.join(folder_path, '*.avi'))
#         csv_list = glob.glob(os.path.join(folder_path, '*.csv'))

#         if not video_list or not csv_list:
#             continue

#         video = video_list[0]
#         csv = csv_list[0]

#         cap = cv.VideoCapture(video)
#         df = pd.read_csv(csv)

#         fps = cap.get(cv.CAP_PROP_FPS)

#         X_feature = []
#         Y_feature = []

#         total_rows = len(df)
#         usable_rows = total_rows - (total_rows % 24)

#         for i in range(usable_rows):
#             timestamp_ms = df["Timestep (ms)"][i]
#             frame_number = int((timestamp_ms / 1000.0) * fps)
#             cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

#             ret, frame = cap.read()
#             if not ret:
#                 break

#             # Convert BGR to RGB for MediaPipe
#             rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

#             results = face_detection.process(rgb)

#             if not results.detections:
#                 continue

#             # Use the first detected face
#             detection = results.detections[0]
#             bbox = detection.location_data.relative_bounding_box

#             h, w, _ = frame.shape
#             x = int(bbox.xmin * w)
#             y = int(bbox.ymin * h)
#             w_box = int(bbox.width * w)
#             h_box = int(bbox.height * h)

#             # Clamp coordinates to image bounds
#             x = max(0, x)
#             y = max(0, y)
#             x2 = min(frame.shape[1], x + w_box)
#             y2 = min(frame.shape[0], y + h_box)

#             green = frame[:, :, 1]
#             face_green = green[y:y2, x:x2]

#             if face_green.size == 0:
#                 continue

#             mean_g = np.mean(face_green)

#             X_feature.append([mean_g])
#             Y_feature.append([df["Heart rate (HR)"][i]])

#         b = int(len(X_feature) / 24)
#         X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)
#         Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

#         X.append(X_feature)
#         Y.append(Y_feature)


In [8]:
len(X)

7

In [9]:
print(len(X[2]))

217


In [10]:
len(X[1])

215

In [11]:
X1 = np.vstack(X)  # or np.concatenate(X, axis=0) depending on shape
Y1 = np.vstack(Y)

np.save("input1_signal.npy", X1)
np.save("output1_signal.npy", Y1)


In [12]:
X1.shape

(1560, 24, 1)

In [13]:
len(X)

7

In [14]:
len(Y)

7

In [15]:
Y

[array([[75, 75, 75, ..., 75, 75, 75],
        [75, 75, 75, ..., 75, 75, 75],
        [75, 75, 75, ..., 75, 75, 75],
        ...,
        [74, 75, 75, ..., 75, 75, 75],
        [75, 75, 75, ..., 75, 75, 75],
        [75, 75, 75, ..., 75, 75, 75]], dtype=int64),
 array([[100, 100, 100, ..., 100, 100, 100],
        [100, 100,  99, ...,  99,  99,  99],
        [ 99,  99,  99, ...,  99,  99,  99],
        ...,
        [ 95,  95,  95, ...,  95,  95,  95],
        [ 95,  95,  95, ...,  95,  95,  95],
        [ 95,  95,  95, ...,  95,  95,  95]], dtype=int64),
 array([[68, 68, 68, ..., 68, 68, 68],
        [68, 68, 68, ..., 68, 68, 68],
        [68, 68, 68, ..., 68, 68, 68],
        ...,
        [82, 82, 82, ..., 82, 82, 82],
        [83, 83, 83, ..., 83, 83, 83],
        [83, 83, 83, ..., 83, 83, 83]], dtype=int64),
 array([[78, 78, 78, ..., 78, 78, 78],
        [78, 78, 78, ..., 78, 78, 78],
        [78, 78, 78, ..., 78, 78, 78],
        ...,
        [81, 81, 81, ..., 81, 81, 80],
        [

In [16]:
out=np.mean(Y1,axis=1).astype("int64")
out=out.reshape(-1,1)

In [17]:
out

array([[ 75],
       [ 75],
       [ 75],
       ...,
       [100],
       [ 99],
       [ 99]], dtype=int64)

In [18]:
np.save("out1_signal.npy", out)

In [19]:
len(out)

1560

In [20]:
len(out)

1560

In [21]:
X1.shape

(1560, 24, 1)

In [22]:
# df=pd.read_csv(r"E:\Dataset-2\subject8\ground_truth.txt",delimiter="/t")

In [23]:
# df

In [24]:
# with open('E:\Dataset-2\subject8\ground_truth.txt', 'r') as f:
#     lines = f.readlines()

# ppg_values = list(map(float, lines[0].strip().split()))
# hr_values = list(map(float, lines[1].strip().split()))
# time_values = list(map(float, lines[2].strip().split()))

In [25]:
# ppg_values

In [26]:
# print(len(time_values))
# print(len(ppg_values))
# print(len(hr_values))



In [27]:
# hr_values

In [28]:
# time_values

In [29]:
input1=np.load("input1_signal.npy")
output1=np.load("out1_signal.npy")

In [31]:
input1.shape

(1560, 24, 1)

In [41]:
unique_vals, counts = np.unique(output1, return_counts=True)

print(unique_vals)  # [1 2 3]
print(counts)       # [1 2 3]
print(np.sum(counts))   

[ 62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 133 134
 135 136 137 138 140 141 143 145 147 148]
[ 18  22   6   7  39   8  15   5  25  40   8  38  23  56  79  49 113  63
  17  29  16  18  25  27   9  19   3  26  21  47  62  35  92  57  35  47
  47  30  15  12  12  29  19  24  20   1   1   1   1   2  11   6   6   3
   7  28   2  10   3   1   2   2   2   4   3   6   5   1   1   2   2   1
   5   1   4   1   4   1   1   5   1  16]
1560
